In [1]:
#Import here
from sentence_transformers import SentenceTransformer, util
import torch
from getDocument import *

c:\Users\zebzi\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from transformers import AutoModel, AutoTokenizer, BertModel

model = BertModel.from_pretrained("allenai/scibert_scivocab_uncased", is_decoder=True) #add_cross_attention
tokenizer = AutoTokenizer.from_pretrained('allenai/scibert_scivocab_uncased', padding=True, truncation=True) 

Some weights of the model checkpoint at allenai/scibert_scivocab_uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


The following is an example of running Torch.nn.CosineSimilarity() on single sentences. It works somewhat as text1 and text2 have a higher similarity than 1/3 and 2/3.  Torch.nn.CosineSimilarity(dim=2) returns a vector of size [1, 14] which I then sum to get the similarity *THIS MAY NOT BE THE CORRECT WAY TO DO IT* but right now I don't have any better guidance.

In [3]:
# Test of sentences to gather understanding of what cosine similarity is
# https://pytorch.org/docs/stable/generated/torch.nn.CosineSimilarity.html

text1 = "No one was allowed to interfere with the work."
text2 = "The job was done with no interaction."
text3 = "Walking in the park is fun when it's warm."

MAX_LENGTH = 18

# Need to do the padding buisness because nn.CosineSimilarity requires tensors of equal length
encoded_input1 = tokenizer(text1, return_tensors='pt', max_length=MAX_LENGTH, padding='max_length')
encoded_input2 = tokenizer(text2, return_tensors='pt', max_length=MAX_LENGTH, padding='max_length')
encoded_input3 = tokenizer(text3, return_tensors='pt', max_length=MAX_LENGTH, padding='max_length')

output1 = model(**encoded_input1, return_dict=False)
output2 = model(**encoded_input2, return_dict=False)
output3 = model(**encoded_input3, return_dict=False)

print(f"encoded_input1: {encoded_input1['input_ids']}")
print(f"encoded_input2: {encoded_input2['input_ids']}")
print(f"encoded_input3: {encoded_input3['input_ids']}")
print(f"decode of encoded_input1: {tokenizer.decode(encoded_input1['input_ids'][0])}")
print(f"decode of encoded_input2: {tokenizer.decode(encoded_input2['input_ids'][0])}")
print(f"decode of encoded_inpu32: {tokenizer.decode(encoded_input3['input_ids'][0])}")

# print(output1[0].shape)
# print(output2[0].shape)
# print(output3[0].shape)

# cosine_scores = util.cos_sim(encoded_input1["input_ids"], encoded_input2["input_ids"])
cosine_scores_DIM2 = torch.nn.CosineSimilarity(dim=2)
# cosine_scores2 = torch.nn.CosineSimilarity(dim=1)
# cosine_scores3 = torch.nn.CosineSimilarity(dim=2)

score1 = cosine_scores_DIM2(output1[0], output2[0])
score2 = cosine_scores_DIM2(output1[0], output3[0])
score3 = cosine_scores_DIM2(output2[0], output3[0])
# score4 = cosine_scores4(output1[0], output2[0])

# print(f"score1.shape: {score1.shape}")
# print(f"score2.shape: {score2.shape}")
# print(f"score3.shape: {score3.shape}")

# print(f"sum of score3: {score3[0].sum().item()} and score3[0].size()[0]: {score3[0].size()[0]}")
print(f"Sentence 1 & 2 cosine similarity score: {score1.mean().item()}")
print(f"Sentence 1 & 3 cosine similarity score: {score2.mean().item()}")
print(f"Sentence 2 & 3 cosine similarity score: {score3.mean().item()}")

encoded_input1: tensor([[  102,   425,   482,   241,  4254,   147, 13780,   190,   111,   697,
           205,   103,     0,     0,     0,     0,     0,     0]])
encoded_input2: tensor([[ 102,  111, 4417,  241, 2992,  190,  425, 2019,  205,  103,    0,    0,
            0,    0,    0,    0,    0,    0]])
encoded_input3: tensor([[  102,  9704,   121,   111,  7713,   165, 11035,   603,   256,  2505,
           112,  8591,   205,   103,     0,     0,     0,     0]])
decode of encoded_input1: [CLS] no one was allowed to interfere with the work. [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]
decode of encoded_input2: [CLS] the job was done with no interaction. [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]
decode of encoded_inpu32: [CLS] walking in the park is fun when it's warm. [SEP] [PAD] [PAD] [PAD] [PAD]
Sentence 1 & 2 cosine similarity score: 0.6555863618850708
Sentence 1 & 3 cosine similarity score: 0.5731323957443237
Sentence 2 & 3 cosine similarity score: 0.5880245566368103


With max_length=14 the scores are
1/2 -> 0.582
1/3 -> 0.475
2/3 -> 0.490

With max_length = 18 the scores are
1/2 -> 0.656
1/3 -> 0.573
2/3 -> 0.588

seeming to suggest that unnecessary padding will hurt the accuracy (as expected)
_____________________________________________________________________________________________________________________

In [4]:
paragraph1 = "Pre-trained text encoders have rapidly advanced the state of the art on many NLPtasks. We focus on one such model, BERT, and aim to quantify where linguistic information is captured within the network. We find that the model represents the steps of the traditional NLP pipeline in an interpretable and localizable way, and that the regions responsible for each step appear in the expected sequence: POS tagging, parsing, NER, semantic roles, then coreference. Qualitative analysis reveals that the model can and often does adjust this pipeline dynamically, revising lower-level decisions on the basis of disambiguating information from higher-level representations."
paragraph2 = "We present a set of experiments to demonstrate that deep recurrent neural networks (RNNs) learn internal representations that capture soft hierarchical notions of syntax from highly varied supervision. We consider four syntax tasks at different depths of the parse tree; for each word, we predict its part of speech as well as the first (parent), second (grandparent) and third level (great-grandparent) constituent labels that appear above it. These predictions are made from representations produced at different depths in networks that are pretrained with one of four objectives: dependency parsing, semantic role labeling, machine translation, or language modeling. In every case, we find a correspondence between network depth and syntactic depth, suggesting that a soft syntactic hierarchy emerges. This effect is robust across all conditions, indicating that the models encode significant amounts of syntax even in the absence of an explicit syntactic training supervision."
paragraph3 = "Language  technology  has  become  pervasive  in  everyday  life. Neural  networks  are  a key component  in  this technology  thanks  to  their ability  to model  large  amounts  of data.  Contrary  to traditional  systems,  models  based  on  deep  neural  networks  (a.k.a. deep  learning) can  be  trained  in  an  end-to-end  fashion  on  input-output  pairs,  such  as  a  sentence  in  one language  and  its translation  in  another  language,  or  a  speech  utterance  and  its  transcription.  The end-to-end  training  paradigm simplifies  the  engineering process  while  giving  the model  flexibility  to  optimize  for  the  desired  task. This,  however,  often  comes  at  the  expense  of model  interpretability:  understanding  the role  of different parts  of the  deep  neural network  is  difficult,  and  such  models  are  sometimes  perceived  as  'black-box',  hindering research  efforts  and  limiting their  utility to  society."

MAX_LENGTH = 175

# Need to do the padding buisness because nn.CosineSimilarity requires tensors of equal length
encoded_long_input1 = tokenizer(paragraph1, return_tensors='pt', max_length=MAX_LENGTH, padding='max_length')
encoded_long_input2 = tokenizer(paragraph2, return_tensors='pt', max_length=MAX_LENGTH, padding='max_length')
encoded_long_input3 = tokenizer(paragraph3, return_tensors='pt', max_length=MAX_LENGTH, padding='max_length')

output_long_1 = model(**encoded_long_input1, return_dict=False)
output_long_2 = model(**encoded_long_input2, return_dict=False)
output_long_3 = model(**encoded_long_input3, return_dict=False)

# print(f"encoded_input1: {encoded_long_input1['input_ids']}")
# print(f"encoded_input2: {encoded_long_input2['input_ids']}")
# print(f"encoded_input3: {encoded_long_input3['input_ids']}")
print(f"decode of encoded_input1: {tokenizer.decode(encoded_long_input1['input_ids'][0])}")
print(f"decode of encoded_input2: {tokenizer.decode(encoded_long_input2['input_ids'][0])}")
print(f"decode of encoded_inpu32: {tokenizer.decode(encoded_long_input3['input_ids'][0])}")

print(output_long_1[0].shape)
print(output_long_2[0].shape)
print(output_long_3[0].shape)

cosine_scores_DIM2 = torch.nn.CosineSimilarity(dim=0)

score_long_1 = cosine_scores_DIM2(output_long_1[0], output_long_2[0])
score_long_2 = cosine_scores_DIM2(output_long_1[0], output_long_3[0])
score_long_3 = cosine_scores_DIM2(output_long_2[0], output_long_3[0])

print(f"Sentence 1 & 2 cosine similarity score: {score_long_1.mean().item()}")
print(f"Sentence 1 & 3 cosine similarity score: {score_long_2.mean().item()}")
print(f"Sentence 2 & 3 cosine similarity score: {score_long_3.mean().item()}")


decode of encoded_input1: [CLS] pre - trained text encoders have rapidly advanced the state of the art on many nlptasks. we focus on one such model, bert, and aim to quantify where linguistic information is captured within the network. we find that the model represents the steps of the traditional nlp pipeline in an interpretable and localizable way, and that the regions responsible for each step appear in the expected sequence : pos tagging, parsing, ner, semantic roles, then coreference. qualitative analysis reveals that the model can and often does adjust this pipeline dynamically, revising lower - level decisions on the basis of disambiguating information from higher - level representations. [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]
decode of enc

Text came from the BERT&ClassicNLP folder with 1: main paper, 2: source1, 3: source2
using their abstracts, source 3 is supposed to be a semi-black sheep source as it's a thesis abstract and not a normal academic paper one

With max_length=175 & dim=2:
Sentence 1 & 2 cosine similarity score: 0.296
Sentence 1 & 3 cosine similarity score: 0.364
Sentence 2 & 3 cosine similarity score: 0.319

With max_length=175 & dim=1:
Sentence 1 & 2 cosine similarity score: 0.224
Sentence 1 & 3 cosine similarity score: 0.248
Sentence 2 & 3 cosine similarity score: 0.237

With max_length=175 & dim=0:
Sentence 1 & 2 cosine similarity score: 0.191
Sentence 1 & 3 cosine similarity score: 0.219
Sentence 2 & 3 cosine similarity score: 0.201

______________________________________________________________________________________________________________________

Get the tuple of SPECTER embeddings of a main paper and all of its references

Both indices of the tuple are lists, 0 is embeddings and 1 is references

The 0 index of each list is of the main paper

The each embedding is just a list of numbers

In [6]:
tupleOfResults = get_paper_and_references_embedding_and_titles_from_query("BERT Rediscovers the Classical NLP Pipeline")

embeddings = tupleOfResults[0]
titles = tupleOfResults[1]

print(titles)

['BERT Rediscovers the Classical NLP Pipeline', 'What do you learn from context? Probing for sentence structure in contextualized word representations', 'Multi-Task Deep Neural Networks for Natural Language Understanding', 'BERT: Pre-training of Deep Bidirectional Transformers for Language Understanding', 'Language Models are Unsupervised Multitask Learners', 'Dissecting Contextual Word Embeddings: Architecture and Representation', 'Targeted Syntactic Evaluation of Language Models', 'Deep RNNs Encode Soft Hierarchical Syntax', 'What you can cram into a single $&!#* vector: Probing sentence embeddings for linguistic properties', 'Collecting Diverse Natural Language Inference Problems for Sentence Representation Evaluation', 'Deep Contextualized Word Representations', 'On internal language representations in deep learning: an analysis of machine translation and speech recognition', 'Improving Language Understanding by Generative Pre-Training', 'Attention is All you Need', 'Semantic Proto

In [14]:
# Test the Cosine Similarity by first creating tensors and comparing the results
newCosineSimilarity = torch.nn.CosineSimilarity(dim=0)
mainPaper = torch.tensor(embeddings[0])
print(f"Main paper title: {titles[0]}")
for i in range(1, len(embeddings)):
    referencePaper = torch.tensor(embeddings[i])
    print(f"Main Paper with embeddings[{i}]: {newCosineSimilarity(mainPaper, referencePaper):.3f} and the reference paper: {titles[i]}")


Main paper title: BERT Rediscovers the Classical NLP Pipeline
Main Paper with embeddings[1]: 0.751 and the reference paper: What do you learn from context? Probing for sentence structure in contextualized word representations
Main Paper with embeddings[2]: 0.701 and the reference paper: Multi-Task Deep Neural Networks for Natural Language Understanding
Main Paper with embeddings[3]: 0.735 and the reference paper: BERT: Pre-training of Deep Bidirectional Transformers for Language Understanding
Main Paper with embeddings[4]: 0.747 and the reference paper: Language Models are Unsupervised Multitask Learners
Main Paper with embeddings[5]: 0.758 and the reference paper: Dissecting Contextual Word Embeddings: Architecture and Representation
Main Paper with embeddings[6]: 0.670 and the reference paper: Targeted Syntactic Evaluation of Language Models
Main Paper with embeddings[7]: 0.735 and the reference paper: Deep RNNs Encode Soft Hierarchical Syntax
Main Paper with embeddings[8]: 0.684 and

With dim=0: (Which is the only allowed cosine similarity )
Main Paper with embeddings[1]: 0.7507858276367188
Main Paper with embeddings[2]: 0.7007443308830261
Main Paper with embeddings[3]: 0.734921395778656
Main Paper with embeddings[4]: 0.7470535039901733
Main Paper with embeddings[5]: 0.7582955956459045
Main Paper with embeddings[6]: 0.6695845127105713
Main Paper with embeddings[7]: 0.7345977425575256
Main Paper with embeddings[8]: 0.6843916773796082
Main Paper with embeddings[9]: 0.7346400618553162
Main Paper with embeddings[10]: 0.7553737759590149
Main Paper with embeddings[11]: 0.6457442045211792
Main Paper with embeddings[12]: 0.7245481610298157
Main Paper with embeddings[13]: 0.6589515209197998
Main Paper with embeddings[14]: 0.6954355835914612
Main Paper with embeddings[15]: 0.6530366539955139
Main Paper with embeddings[16]: 0.5189672708511353
Main Paper with embeddings[17]: 0.6884840726852417
Main Paper with embeddings[18]: 0.616716206073761
Main Paper with embeddings[19]: 0.644980251789093
Main Paper with embeddings[20]: 0.6220344305038452